In [1]:
import pandas as pd # thư viện đọc dữ liệu ra dạng bảng
import numpy as np
import psycopg2     # thư viện kết nối database

ModuleNotFoundError: No module named 'psycopg2'

In [5]:
# thông tin kết nối database
host = '101.96.116.82'
port = '8182'
database = 'tintuc_haisan'
username = 'mp_quantri'
password = 'asdaw@23423das'

In [6]:
# thực hiện kết nối thông qua thư viện psycopg2
connect = psycopg2.connect(host=host, port=port, dbname=database, user=username, password=password)
cursor = connect.cursor()

In [22]:
table = 'hungpq_mpg_product_price_ring'

query = f'''
SELECT *
FROM {table}
'''

df = pd.read_sql_query(query, connect)
df.columns = df.columns.str.replace('std_', '').str.title()
df.to_excel(table+'.xlsx', index=False)

In [24]:
def number_extract(x):
    output = ''
    if sum(i.isdigit() for i in x) == 0:
        output = str(1e6)
    else:
        for i in range(len(x)):
            if x[i].isdigit():
                output += x[i]
            elif (x[i]=='.' or x[i]==',') and x[i-1].isdigit():
                output += '.'
            else:
                output += str()
            
    numbers = output.split()
    numbers = [float(i) for i in numbers]
    return tuple(numbers)

df['std_size'] = pd.Categorical(
    df.std_size,
    categories=np.array(sorted(df.std_size.unique(), key=number_extract))
)
df = df.drop(columns='std_active_date')
df['std_factory'] = df.std_factory.map({'Hậu Giang': 'MPHG', 'Cà Mau': 'MPCM'})
df.std_type = df.std_type.str.capitalize()
df = df.sort_values(['std_factory', 'std_type', 'std_size'])
df.head()

,std_factory,std_type,std_size,std_pending_interval,std_quantity_pending,std_purchased_interval,std_quantity_purchased
476,MPCM,Sú nguyên con,2/4,2020-07-23 đến 2020-11-27,0.0,2020-07-01 đến 2020-07-23,0.0
782,MPCM,Sú nguyên con,2/4,2020-07-23 đến 2020-11-27,0.0,2020-06-01 đến 2020-06-30,2777.0
111,MPCM,Sú nguyên con,4/6,2020-07-23 đến 2020-11-27,0.0,2020-06-01 đến 2020-06-30,357369.0
735,MPCM,Sú nguyên con,4/6,2020-07-23 đến 2020-11-27,0.0,2020-07-01 đến 2020-07-23,176515.0
438,MPCM,Sú nguyên con,6/7,2020-07-23 đến 2020-11-27,0.0,2020-06-01 đến 2020-06-30,300497.0


In [25]:
df = df.rename(columns={
    'std_size': 'Kích cỡ',
    'std_pending_interval': 'Thời gian nợ',
    'std_quantity_pending': 'Khối lượng nợ',
    'std_purchased_interval': 'Thời gian mua',
    'std_quantity_purchased': 'Khối lượng mua'
})
df.head()

,std_factory,std_type,Kích cỡ,Thời gian nợ,Khối lượng nợ,Thời gian mua,Khối lượng mua
476,MPCM,Sú nguyên con,2/4,2020-07-23 đến 2020-11-27,0.0,2020-07-01 đến 2020-07-23,0.0
782,MPCM,Sú nguyên con,2/4,2020-07-23 đến 2020-11-27,0.0,2020-06-01 đến 2020-06-30,2777.0
111,MPCM,Sú nguyên con,4/6,2020-07-23 đến 2020-11-27,0.0,2020-06-01 đến 2020-06-30,357369.0
735,MPCM,Sú nguyên con,4/6,2020-07-23 đến 2020-11-27,0.0,2020-07-01 đến 2020-07-23,176515.0
438,MPCM,Sú nguyên con,6/7,2020-07-23 đến 2020-11-27,0.0,2020-06-01 đến 2020-06-30,300497.0


In [26]:
group = df.groupby(['std_factory', 'std_type'])

In [33]:
writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')
for name, table in group:
    table = table.drop(columns=['std_factory', 'std_type'])
    table.to_excel(writer, sheet_name=' '.join(name), index=False)
writer.save()